In [1]:
import json

from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp

import matplotlib.pyplot as plt

from pepbench.io import load_challenge_results_from_folder
from pepbench.io import convert_hz_to_ms
from pepbench.datasets import EmpkinsDataset, GuardianDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
root_path = Path("../../")

In [3]:
result_path = root_path.joinpath("results")
result_path

WindowsPath('../../results')

In [4]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
print(empkins_base_path)

\Users\sebas\Development\ResearchInternship\Data\2024_08_PEP_Benchmarking\EmpkinS_Dataset


In [5]:
save_results = True

In [6]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=True, only_labeled=True)
fs_empkins = dataset_empkins.sampling_rate_icg
print(f"Sampling rate ICG: {fs_empkins}")
dataset_guardian = GuardianDataset(guardian_base_path, use_cache=True, only_labeled=True)
fs_guardian = dataset_guardian.sampling_rate_icg
print(f"Sampling rate ICG: {fs_guardian}")

Sampling rate ICG: 1000
Sampling rate ICG: 500


In [7]:
algo_levels = ["q_wave_algorithm", "b_point_algorithm", "outlier_correction_algorithm"]
algo_level_mapping = dict(zip(algo_levels, ["Q-Wave Algorithm", "B-Point Algorithm", "Outlier Correction"]))

In [8]:
results_empkins = load_challenge_results_from_folder(
    result_path.joinpath("empkins_dataset_reference_b_point").resolve(), index_cols_per_sample=["participant", "condition", "phase"]
).per_sample
results_guardian = load_challenge_results_from_folder(
    result_path.joinpath("guardian_dataset_reference_b_point").resolve(), index_cols_per_sample=["participant", "phase"]
).per_sample

In [9]:
results_empkins = results_empkins.droplevel("condition")

In [10]:
results_guardian

heartbeat_id  \
                                                                                            estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant phase                      
forounzafar2018   b-point-reference none                         GDN0005     Pause    0             0   
                                                                                      1             1   
                                                                                      2             2   
                                                                                      3             3   
                                                                                      4             4   
...                                                                                               ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     TiltDown 51           51   
                                                                                      52           52   
                                                                                      53           53   
                                                                                      54           54   
                                                                                      55           55   

                                                                                                    \
                                                                                         reference   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant phase                   
forounzafar2018   b-point-reference none                         GDN0005     Pause    0          0   
                                                                                      1          1   
                                                                                      2          2   
                                                                                      3          3   
                                                                                      4          4   
...                                                                                            ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     TiltDown 51        51   
                                                                                      52        52   
                                                                                      53        53   
                                                                                      54        54   
                                                                                      55        55   

                                                                                         heartbeat_start_sample  \
                                                                                                      estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant phase                                
forounzafar2018   b-point-reference none                         GDN0005     Pause    0                     352   
                                                                                      1                     808   
                                                                                      2                    1252   
                                                                                      3                    1686   
                                                                                      4                    2116   
...                                                                                                         ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     TiltDown 51                  27029   
                                       

In [11]:
results_empkins_norm = results_empkins.copy()
results_guardian_norm = results_guardian.copy()
results_empkins_norm[("q_wave_onset_sample", "estimated")] = results_empkins[("q_wave_onset_sample", "estimated")] - results_empkins[("heartbeat_start_sample", "estimated")]
results_empkins_norm[("q_wave_onset_sample", "reference")] = results_empkins[("q_wave_onset_sample", "reference")] - results_empkins[("heartbeat_start_sample", "estimated")]
results_guardian_norm[("q_wave_onset_sample", "estimated")] = results_guardian[("q_wave_onset_sample", "estimated")] - results_guardian[("heartbeat_start_sample", "estimated")]
results_guardian_norm[("q_wave_onset_sample", "reference")] = results_guardian[("q_wave_onset_sample", "reference")] - results_guardian[("heartbeat_start_sample", "estimated")]

In [12]:
results_guardian_norm

heartbeat_id  \
                                                                                            estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant phase                      
forounzafar2018   b-point-reference none                         GDN0005     Pause    0             0   
                                                                                      1             1   
                                                                                      2             2   
                                                                                      3             3   
                                                                                      4             4   
...                                                                                               ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     TiltDown 51           51   
                                                                                      52           52   
                                                                                      53           53   
                                                                                      54           54   
                                                                                      55           55   

                                                                                                    \
                                                                                         reference   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant phase                   
forounzafar2018   b-point-reference none                         GDN0005     Pause    0          0   
                                                                                      1          1   
                                                                                      2          2   
                                                                                      3          3   
                                                                                      4          4   
...                                                                                            ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     TiltDown 51        51   
                                                                                      52        52   
                                                                                      53        53   
                                                                                      54        54   
                                                                                      55        55   

                                                                                         heartbeat_start_sample  \
                                                                                                      estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant phase                                
forounzafar2018   b-point-reference none                         GDN0005     Pause    0                     352   
                                                                                      1                     808   
                                                                                      2                    1252   
                                                                                      3                    1686   
                                                                                      4                    2116   
...                                                                                                         ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     TiltDown 51                  27029   
                                       

In [13]:
cols = [("heartbeat_start_sample", "estimated"), ("heartbeat_start_sample", "reference"), ("heartbeat_end_sample", "estimated"), ("heartbeat_end_sample", "reference"), ("q_wave_onset_sample", "estimated"), ("q_wave_onset_sample", "reference"), ("b_point_sample", "estimated"), ("b_point_sample", "reference"), ("pep_sample", "estimated"), ("pep_sample", "reference")]
results_empkins_cleaned_ms = results_empkins_norm.copy()
results_guardian_cleaned_ms = results_guardian_norm.copy()
results_empkins_cleaned_ms[cols] = results_empkins_norm[cols] * convert_hz_to_ms(sampling_frequency=fs_empkins)
results_guardian_cleaned_ms[cols] = results_guardian_norm[cols] * convert_hz_to_ms(sampling_frequency=fs_guardian)

In [14]:
results_guardian_cleaned_ms

heartbeat_id  \
                                                                                            estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant phase                      
forounzafar2018   b-point-reference none                         GDN0005     Pause    0             0   
                                                                                      1             1   
                                                                                      2             2   
                                                                                      3             3   
                                                                                      4             4   
...                                                                                               ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     TiltDown 51           51   
                                                                                      52           52   
                                                                                      53           53   
                                                                                      54           54   
                                                                                      55           55   

                                                                                                    \
                                                                                         reference   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant phase                   
forounzafar2018   b-point-reference none                         GDN0005     Pause    0          0   
                                                                                      1          1   
                                                                                      2          2   
                                                                                      3          3   
                                                                                      4          4   
...                                                                                            ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     TiltDown 51        51   
                                                                                      52        52   
                                                                                      53        53   
                                                                                      54        54   
                                                                                      55        55   

                                                                                         heartbeat_start_sample  \
                                                                                                      estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant phase                                
forounzafar2018   b-point-reference none                         GDN0005     Pause    0                   704.0   
                                                                                      1                  1616.0   
                                                                                      2                  2504.0   
                                                                                      3                  3372.0   
                                                                                      4                  4232.0   
...                                                                                                         ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     TiltDown 51                54058.0   
                                       

In [15]:
preprocessed_results_empkins_guardian = pd.concat([results_empkins_cleaned_ms, results_guardian_cleaned_ms])
preprocessed_results_empkins_guardian

heartbeat_id  \
                                                                                            estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant phase                      
forounzafar2018   b-point-reference none                         VP_001      Prep     0             0   
                                                                                      1             1   
                                                                                      2             2   
                                                                                      3             3   
                                                                                      4             4   
...                                                                                               ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     TiltDown 51           51   
                                                                                      52           52   
                                                                                      53           53   
                                                                                      54           54   
                                                                                      55           55   

                                                                                                    \
                                                                                         reference   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant phase                   
forounzafar2018   b-point-reference none                         VP_001      Prep     0          0   
                                                                                      1          1   
                                                                                      2          2   
                                                                                      3          3   
                                                                                      4          4   
...                                                                                            ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     TiltDown 51        51   
                                                                                      52        52   
                                                                                      53        53   
                                                                                      54        54   
                                                                                      55        55   

                                                                                         heartbeat_start_sample  \
                                                                                                      estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant phase                                
forounzafar2018   b-point-reference none                         VP_001      Prep     0                   394.0   
                                                                                      1                   984.0   
                                                                                      2                  1569.0   
                                                                                      3                  2134.0   
                                                                                      4                  2684.0   
...                                                                                                         ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     TiltDown 51                54058.0   
                                       

### Check whether the concatendated dataframe has the same size as the summed non-concatenated dataframes

In [16]:
print(preprocessed_results_empkins_guardian.shape[0] - (results_empkins_cleaned_ms.shape[0] + results_guardian_cleaned_ms.shape[0]))

0


In [17]:
input_data = preprocessed_results_empkins_guardian

In [18]:
input_data = input_data[[("heartbeat_id", "reference"), ("q_wave_onset_sample", "estimated"), ("q_wave_onset_sample", "reference"), ("rr_interval_ms", "estimated")]]
input_data.columns = input_data.columns.to_flat_index().str.join("_")
input_data

heartbeat_id_reference  \
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant phase                                 
forounzafar2018   b-point-reference none                         VP_001      Prep     0                        0   
                                                                                      1                        1   
                                                                                      2                        2   
                                                                                      3                        3   
                                                                                      4                        4   
...                                                                                                          ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     TiltDown 51                      51   
                                                                                      52                      52   
                                                                                      53                      53   
                                                                                      54                      54   
                                                                                      55                      55   

                                                                                          q_wave_onset_sample_estimated  \
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant phase                                        
forounzafar2018   b-point-reference none                         VP_001      Prep     0                           187.0   
                                                                                      1                           186.0   
                                                                                      2                           182.0   
                                                                                      3                           174.0   
                                                                                      4                           170.0   
...                                                                                                                 ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     TiltDown 51                          338.0   
                                                                                      52                          336.0   
                                                                                      53                          346.0   
                                                                                      54                          324.0   
                                                                                      55                          316.0   

                                                                                          q_wave_onset_sample_reference  \
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant phase                                        
forounzafar2018   b-point-reference none                         VP_001      Prep     0                           173.0   
                                                                                      1                           172.0   
                                                                                      2                           166.0   
                                                                                      3                           157.0   
                                                                                      4                           162.0   
...                                                                                                                 ... 

In [19]:
input_data.xs(key='VP_022', level='participant').max()

heartbeat_id_reference            39.0
q_wave_onset_sample_estimated    312.0
q_wave_onset_sample_reference    299.0
rr_interval_ms_estimated         952.0
dtype: float64

In [20]:
# Pivot the DataFrame
input_data_wide = input_data.pivot_table(
    index=['participant', 'phase', 'heartbeat_id_reference', 'q_wave_onset_sample_reference', 'rr_interval_ms_estimated'],
    columns="q_wave_algorithm",
    values='q_wave_onset_sample_estimated'
).reset_index()

# Flatten the columns
input_data_wide.columns.name = None
input_data_wide.columns = [f'{col}' if isinstance(col, str) else f'{col[1]}' for col in input_data_wide.columns]
input_data_wide

participant          phase  heartbeat_id_reference  \
0         GDN0005  HoldingBreath                       0   
1         GDN0005  HoldingBreath                       1   
2         GDN0005  HoldingBreath                       3   
3         GDN0005  HoldingBreath                       4   
4         GDN0005  HoldingBreath                       5   
...           ...            ...                     ...   
11499      VP_032           Talk                      39   
11500      VP_032           Talk                      40   
11501      VP_032           Talk                      41   
11502      VP_032           Talk                      42   
11503      VP_032           Talk                      43   

       q_wave_onset_sample_reference  rr_interval_ms_estimated  \
0                              260.0                     850.0   
1                              260.0                     778.0   
2                              222.0                     746.0   
3                              220.0                     766.0   
4                              228.0                     790.0   
...                              ...                       ...   
11499                          172.0                     619.0   
11500                          181.0                     680.0   
11501                          200.0                     719.0   
11502                          212.0                     698.0   
11503                          207.0                     689.0   

       forounzafar2018  martinez2004  scipy-findpeaks  vanlien2013-32-ms  \
0                276.0         262.0            262.0              266.0   
1                278.0         262.0            262.0              266.0   
2                236.0         224.0            224.0              230.0   
3                236.0         222.0            222.0              230.0   
4                242.0         230.0            230.0              236.0   
...                ...           ...              ...                ...   
11499            183.0         173.0            173.0              177.0   
11500            193.0         182.0            182.0              185.0   
11501            210.0         201.0            201.0              206.0   
11502            223.0         213.0            213.0              220.0   
11503            217.0           NaN              NaN              212.0   

       vanlien2013-34-ms  vanlien2013-36-ms  vanlien2013-38-ms  \
0                  264.0              262.0              260.0   
1                  264.0              262.0              260.0   
2                  228.0              226.0              224.0   
3                  228.0              226.0              224.0   
4                  234.0              232.0              230.0   
...                  ...                ...                ...   
11499              175.0              173.0              171.0   
11500              183.0              181.0              179.0   
11501              204.0              202.0              200.0   
11502              218.0              216.0              214.0   
11503              210.0              208.0              206.0   

       vanlien2013-40-ms  vanlien2013-42-ms  
0                  258.0              256.0  
1                  258.0              256.0  
2                  222.0              220.0  
3                  222.0              220.0  
4                  228.0              226.0  
...                  ...                ...  
11499              169.0              167.0  
11500              177.0              175.0  
11501              198.0              196.0  
11502              212.0              210.0  
11503              204.0              202.0  

[11504 rows x 14 columns]

In [21]:
input_data_wide.isna().any()

participant                      False
phase                            False
heartbeat_id_reference           False
q_wave_onset_sample_reference    False
rr_interval_ms_estimated         False
forounzafar2018                   True
martinez2004                      True
scipy-findpeaks                   True
vanlien2013-32-ms                False
vanlien2013-34-ms                False
vanlien2013-36-ms                False
vanlien2013-38-ms                False
vanlien2013-40-ms                False
vanlien2013-42-ms                False
dtype: bool

In [22]:
input_data_wide = input_data_wide.dropna()
input_data_wide

participant          phase  heartbeat_id_reference  \
0         GDN0005  HoldingBreath                       0   
1         GDN0005  HoldingBreath                       1   
2         GDN0005  HoldingBreath                       3   
3         GDN0005  HoldingBreath                       4   
4         GDN0005  HoldingBreath                       5   
...           ...            ...                     ...   
11498      VP_032           Talk                      38   
11499      VP_032           Talk                      39   
11500      VP_032           Talk                      40   
11501      VP_032           Talk                      41   
11502      VP_032           Talk                      42   

       q_wave_onset_sample_reference  rr_interval_ms_estimated  \
0                              260.0                     850.0   
1                              260.0                     778.0   
2                              222.0                     746.0   
3                              220.0                     766.0   
4                              228.0                     790.0   
...                              ...                       ...   
11498                          153.0                     596.0   
11499                          172.0                     619.0   
11500                          181.0                     680.0   
11501                          200.0                     719.0   
11502                          212.0                     698.0   

       forounzafar2018  martinez2004  scipy-findpeaks  vanlien2013-32-ms  \
0                276.0         262.0            262.0              266.0   
1                278.0         262.0            262.0              266.0   
2                236.0         224.0            224.0              230.0   
3                236.0         222.0            222.0              230.0   
4                242.0         230.0            230.0              236.0   
...                ...           ...              ...                ...   
11498            188.0         154.0            154.0              179.0   
11499            183.0         173.0            173.0              177.0   
11500            193.0         182.0            182.0              185.0   
11501            210.0         201.0            201.0              206.0   
11502            223.0         213.0            213.0              220.0   

       vanlien2013-34-ms  vanlien2013-36-ms  vanlien2013-38-ms  \
0                  264.0              262.0              260.0   
1                  264.0              262.0              260.0   
2                  228.0              226.0              224.0   
3                  228.0              226.0              224.0   
4                  234.0              232.0              230.0   
...                  ...                ...                ...   
11498              177.0              175.0              173.0   
11499              175.0              173.0              171.0   
11500              183.0              181.0              179.0   
11501              204.0              202.0              200.0   
11502              218.0              216.0              214.0   

       vanlien2013-40-ms  vanlien2013-42-ms  
0                  258.0              256.0  
1                  258.0              256.0  
2                  222.0              220.0  
3                  222.0              220.0  
4                  228.0              226.0  
...                  ...                ...  
11498              171.0              169.0  
11499              169.0              167.0  
11500              177.0              175.0  
11501              198.0              196.0  
11502              212.0              210.0  

[11349 rows x 14 columns]

In [23]:
print(f"Save results: {save_results}")

Save results: True


In [24]:
if save_results:
    input_data_wide.to_csv(result_path.joinpath("data/train_data_q_wave.csv"))